# Post‑Quantum Hash Function ‑ Demo Notebook
This notebook reproduces the toy example, data generation, preprocessing, and LSTM‑based pre‑image attack that our team constructed.

Based on the research paper *“Post‑quantum hash functions using $\mathrm{SL}_n(\mathbb F_p)$”* written by Corentin Le Coz, Christopher Battarbee, Ramon Flores, Thomas Koberda, and Delaram Kahrobaei.

## 1. Constructing the toy hash instance
We follow the concrete parameters from Section 2.4 of the paper: $n=3$, $a=4$, $b=2$, walk length $\ell=4$ and prime $p=5$.
The helper functions build the matrices $A$ and $B$, raise them to power $\ell$, take inverses, and map the IDs 1, 2, 3 as described.

In [1]:
import numpy as np

def create_matrices(n, a, b):
    A = np.eye(n)
    B = np.eye(n)
    for i in range(n-1):
        A[i, i+1] = a
        B[i+1, i] = b
    return A, B

def matrix_power(M, l): 
    return np.linalg.matrix_power(M, l)

def inverse_matrix(M):  
    return np.linalg.inv(M)

def compute_hash(sequence, mapping):
    result = np.eye(mapping[1].shape[0])
    for number in sequence:
        result = np.dot(result, mapping[number])
    return result

# toy parameters
n,a,b,l = 3,4,2,4
A,B = create_matrices(n,a,b)
A_l, B_l = matrix_power(A,l), matrix_power(B,l)
# create the mapping
A_inv, B_inv = inverse_matrix(A_l), inverse_matrix(B_l)
mapping = {
    1: B_l,
    2: A_inv,
    3: B_inv
}
sequence = [2,2,3,2,2,2,1]
result = compute_hash(sequence, mapping)
print(result)

[[ 6.94190977e+08  2.33260720e+08  2.92979520e+07]
 [-3.83796480e+07 -1.28962550e+07 -1.61979200e+06]
 [ 1.19193600e+06  4.00512000e+05  5.03050000e+04]]


> The printed matrix matches the concrete example in the paper, confirming the construction.

## 2. Generating all sequences of length 8 and their hashes
We enumerate all $3^8$ sequences over the alphabet {1,2,3}, compute the hash mod $p=5$, and store flattened outputs.

In [2]:
import itertools, numpy as np, torch
p = 5

def hash_mod(seq_str):
    seq_ints = [int(ch) for ch in seq_str]
    return np.mod(compute_hash(seq_ints, mapping), p)

seqs = [''.join(s) for s in itertools.product('123', repeat=8)]
inputs = np.array(seqs)
outputs = np.array([hash_mod(s).flatten() for s in seqs])

np.save('all_inputs.npy', inputs)
np.save('all_outputs.npy', outputs)
print(f"Saved {len(seqs):,} input–output pairs.")

Saved 6,561 input–output pairs.


## 3. Preprocessing
Convert character sequences to integer tokens and build a `TensorDataset` for PyTorch.

In [3]:
import torch
token_inputs = [[int(ch) for ch in seq] for seq in inputs]
X = torch.tensor(token_inputs, dtype=torch.long)
y = torch.tensor(outputs, dtype=torch.float32)
dataset = torch.utils.data.TensorDataset(X, y)
torch.save(dataset, 'dataset.pth')
print('TensorDataset saved with shapes', X.shape, y.shape)

TensorDataset saved with shapes torch.Size([6561, 8]) torch.Size([6561, 9])


## 4. LSTM model and training
We use a single‑layer LSTM (hidden size 100) followed by a linear head, trained with MSE.

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split

# Load full dataset and unpack tensors
dataset = torch.load('dataset.pth', weights_only=False)
X_all, y = dataset.tensors

# Train: 50%, Probe: 50%
probe_frac = 0.50
probe_sz = int(len(dataset) * probe_frac)
train_sz = len(dataset) - probe_sz
train_ds, probe_ds = random_split(dataset, [train_sz, probe_sz], generator=torch.Generator().manual_seed(42))
print(f"Train size: {train_sz}, Probe size: {probe_sz}, Total: {len(dataset)}")

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
probe_loader = DataLoader(probe_ds, batch_size=32)

# Define the LSTM model
class HashPredictor(nn.Module):
    def __init__(self, vocab=4, embed=32, hidden=100, out_dim=None):
        super().__init__()
        self.embed = nn.Embedding(vocab, embed)
        self.lstm = nn.LSTM(embed, hidden, batch_first=True)
        self.fc = nn.Linear(hidden, out_dim)

    def forward(self, x):
        x = self.embed(x)
        _, (h, _) = self.lstm(x)
        return self.fc(h.squeeze(0))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HashPredictor(out_dim=y.shape[1]).to(device)
opt = optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

# Training loop
for epoch in range(40):
    model.train()
    epoch_loss = 0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        opt.zero_grad()
        loss = loss_fn(model(xb), yb)
        loss.backward()
        opt.step()
        epoch_loss += loss.item() * xb.size(0)
    print(f'Epoch {epoch+1:02d} train MSE {epoch_loss/train_sz:.4f}')

Train size: 3281, Probe size: 3280, Total: 6561
Epoch 01 train MSE 2.4545
Epoch 02 train MSE 1.9083
Epoch 03 train MSE 1.8967
Epoch 04 train MSE 1.8729
Epoch 05 train MSE 1.8231
Epoch 06 train MSE 1.7684
Epoch 07 train MSE 1.7436
Epoch 08 train MSE 1.7335
Epoch 09 train MSE 1.7261
Epoch 10 train MSE 1.7161
Epoch 11 train MSE 1.7020
Epoch 12 train MSE 1.6644
Epoch 13 train MSE 1.6241
Epoch 14 train MSE 1.5787
Epoch 15 train MSE 1.5373
Epoch 16 train MSE 1.4915
Epoch 17 train MSE 1.4509
Epoch 18 train MSE 1.4048
Epoch 19 train MSE 1.3643
Epoch 20 train MSE 1.3200
Epoch 21 train MSE 1.2805
Epoch 22 train MSE 1.2371
Epoch 23 train MSE 1.2016
Epoch 24 train MSE 1.1636
Epoch 25 train MSE 1.1319
Epoch 26 train MSE 1.0968
Epoch 27 train MSE 1.0710
Epoch 28 train MSE 1.0371
Epoch 29 train MSE 1.0093
Epoch 30 train MSE 0.9803
Epoch 31 train MSE 0.9525
Epoch 32 train MSE 0.9252
Epoch 33 train MSE 0.8988
Epoch 34 train MSE 0.8760
Epoch 35 train MSE 0.8527
Epoch 36 train MSE 0.8276
Epoch 37 train M

## 5. Evaluating search‑space reduction
Rank only the probe candidates and compute the average fraction of the space the attacker can skip.

In [5]:
probe_inputs, probe_hashes = probe_ds[:]
probe_inputs, probe_hashes = probe_inputs.to(device), probe_hashes.to(device)

model.eval()
with torch.no_grad():
    all_preds = model(X_all.to(device))
    probe_preds = model(probe_inputs)
    mse_probe = nn.functional.mse_loss(probe_preds, probe_hashes)
    
print(f"Probe set MSE: {mse_probe.item():.4f}")

reductions = []
for t_input, t_hash in zip(probe_inputs, probe_hashes):
    t_input = t_input.unsqueeze(0)
    t_hash = t_hash.unsqueeze(0)
    dists = torch.norm(all_preds - t_hash, dim=1)
    true_index = (X_all == t_input.squeeze(0)).all(dim=1).nonzero(as_tuple=False).item()
    rank = torch.argsort(dists).tolist().index(true_index) + 1
    reductions.append(1 - rank / len(X_all))

print(f'\nAverage search-space reduction on unseen probe set: {100 * torch.tensor(reductions).mean():.2f}%')

Probe set MSE: 0.9053

Average search-space reduction on unseen probe set: 93.88%
